Predicting Movie Tags from Plots using Gensim's Doc2Vec 
по мотивам http://sujitpal.blogspot.ru/2016/04/predicting-movie-tags-from-plots-using.html
Задача https://jira.rutube.ru/browse/STAT-744
Размеченность роликов https://jira.rutube.ru/browse/STAT-679
* тэги - 92%
* хэштэги - 15%

In [1]:
# -*- coding: utf-8 -*-
# Source: src/build_dataset.py
import pymysql
import numpy
import pandas as pd
conn=pymysql.connect(host="10.9.200.25",user="alukyanov", passwd="eifAqyjY",db="squirrel"
                     ,charset='utf8', use_unicode=True)
cur = conn.cursor()
selectSQL = """
select -- count(*)
        v.track_id -- , t2.video_id
        , t2.tags -- video_category.name tags -- 
        , case when length(video_videoeditorsmeta.description)>0 then video_videoeditorsmeta.description else v.description end description
from squirrel.video_video v
        left join video_videoeditorsmeta on video_videoeditorsmeta.video_id = v.id 
        join  
        (
        select hv.object_id video_id, GROUP_CONCAT(dir.name SEPARATOR '|') tags 
        from squirrel.tags_taggeditem hv
                join squirrel.tags_tag dir on dir.id = hv.tag_id
        group by 1          
        ) t2 on t2.video_id = v.id
    -- join squirrel.tags_hashtag_videos hv on hv.video_id = v.id
        -- join squirrel.tags_hashtag dir on dir.id = v.hashtag_id      
        join squirrel.auth_user on auth_user.id = v.author_id  
              left join user_vip on auth_user.id = user_vip.squirrel_user_id
        -- left join video_category on video_category.id = v.category_id                                         
where
    (length(video_videoeditorsmeta.description)>20 or length(v.description)>20)
    and user_vip.licensed_content
    and not v.is_deleted
    and not v.is_hidden
order by 1 desc
limit 2000000 -- 178480 rows -- 152472
"""

df = pd.read_sql(selectSQL, con=conn)
cur.close()
conn.close()
# df['tags'] = df['tags'].str.decode('unicode-escape', errors='replace') # ignore
df

,track_id,tags,description
0,10876671,Hi-tech|Новости,В США битва за так называемый сетевой нейтрали...
1,10876670,Дзен. Нарезки|Новости,Первое автоспортивное соревнование года 21-ая ...
2,10876649,Новости|Дзен. Нарезки,Экипаж Эдуарда Николаева из российской команды...
3,10876636,Новости|Дзен. Нарезки,В Актюбинской области Казахстана сгорел автобу...
4,10876627,Дзен. Нарезки|Новости,"В Москву сегодня вернулись россияне, которые б..."
5,10876609,Владимир Путин|Дзен. Нарезки|Новости,18 января отметят 75-ую годовщину начала проры...
6,10876599,Новости|Дзен. Нарезки|Дональд Трамп,"Дональд Трамп объявил победителей конкурса ""фе..."
7,10876597,Дзен. Нарезки|Новости,"В Якутии после минус 65 ""потеплело"". Сейчас в..."
8,10876593,Новости|Дзен. Нарезки|Мария Шарапова,В Мельбурне продолжается Открытый чемпионат Ав...
9,10876586,Hi-tech|Hi-Tech NEW|Новости,Борьба цифровых ассистентов. Google показала н...


In [2]:
# Source: src/doc2vec.py
# -*- coding: utf-8 -*-
from __future__ import division
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from random import shuffle
from sklearn.cross_validation import train_test_split # model_selection
import nltk
import numpy as np

def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens
    
def tokenize_tags(label):
    tags = label.split("|")
#     for tag in tags:      
    tags = list(map(lambda tok: mark_tag(tok), tags))
    return tags
#     tags = label.split("|")
#     for sent in tags:
#         tags
#     return tokens

def mark_tag(s):
    return "_" + s.replace(" ", "_")
    
def unmark_tag(s):
    return s[1:].replace("_", " ")
    
# read input data
orig_sents = []
sentences = []
# fdata = open("../data/tagged_plots.csv", 'rb')
for index, row in df.iterrows(): # for row in df: # for line in fdata:
    mid, text, label = row["track_id"], row["description"], row["tags"] # row[0], row[2], row[1] #  line.strip().split("\t")
#     print(label)
    orig_sents.append(text)
    tokens = tokenize_text(text)
    tags = tokenize_tags(label)
    sentences.append(TaggedDocument(words=tokens, tags=tags))
# fdata.close()


# Split model into 90/10 training and test
train_sents, test_sents = train_test_split(sentences, test_size=0.1, 
                                           random_state=42) 

## Build and train model

## PV-DM w/concatenation
#model = Doc2Vec(dm=1, dm_concat=1, size=100, window=5, negative=5, 
#                hs=0, min_count=2)

## PV-DM w/averaging
#model = Doc2Vec(dm=1, dm_mean=1, size=100, window=5, negative=5, 
#                hs=0, min_count=2)                

# PV-DBOW
model = Doc2Vec(dm=0, size=100, negative=5, hs=0, min_count=2
               , workers=8, sample=1e-5) #! , total_examples=model.corpus_count

model.build_vocab(sentences)

alpha = 0.025
min_alpha = 0.001
num_epochs = 20
alpha_delta = (alpha - min_alpha) / num_epochs

for epoch in range(num_epochs):
    shuffle(sentences)
    model.alpha = alpha
    model.min_alpha = alpha
#     model.total_examples=model.corpus_count #    print(model.corpus_count)
    model.train(sentences, total_examples=model.corpus_count, epochs=model.iter)
    alpha -= alpha_delta
# model_d2v = Doc2Vec(alpha=0.025, min_alpha=0.025, size=50, window=5, min_count=5,
#                 dm=1, workers=8, sample=1e-5)        

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
# model_d2v_TEST = Doc2Vec.load('./variants/01_all_license/model.d2v')

In [4]:
# evaluate the model
# Коэффициент Жаккара
def jaccard_similarity(labels, preds):
    labels_set = frozenset(labels)
    preds_set = frozenset(preds)
    return len(labels_set.intersection(preds_set)) / len(labels_set.union(preds_set))

# вхождение предсказаных тэгов в топ (5) относительно фактических (эксперементальная метрика) 
def rate_similarity(labels, preds):
    labels_set = frozenset(labels)
    preds_set = frozenset(preds)
    return len(labels_set.intersection(preds_set)) / len(labels_set)

total_sim_jaccard = 0.0
total_sim_rate = 0.0
for test_sent in test_sents:
    pred_vec = model.infer_vector(test_sent.words)
    actual_tags = list(map(lambda x: unmark_tag(x), test_sent.tags))
    pred_tags = model.docvecs.most_similar([pred_vec], topn=5)
    pred_tags = filter(lambda x: x[0].find("_") > -1, pred_tags)
    pred_tags = list(map(lambda x: (unmark_tag(x[0]), x[1]), pred_tags))
    total_sim_jaccard += jaccard_similarity(actual_tags, [x[0] for x in pred_tags])     
    total_sim_rate += rate_similarity(actual_tags, [x[0] for x in pred_tags])
print ("Average Similarity on Test Set (jaccard): %.3f" % (total_sim_jaccard / len(test_sents)))
print ("Average Similarity on Test Set (intersection of top over actual): %.3f" % (total_sim_rate / len(test_sents)))

Average Similarity on Test Set (jaccard): 0.198
Average Similarity on Test Set (intersection of top over actual): 0.720


In [5]:
# print out random test result
for i in range(5):
    docid = np.random.randint(len(sentences))
    pred_vec = model.infer_vector(sentences[docid].words)
    actual_tags = map(lambda x: unmark_tag(x), sentences[docid].tags)
    pred_tags = model.docvecs.most_similar([pred_vec], topn=5)
    print ("TEXT: %s" % (orig_sents[docid][0:1220]))
    print ("... Actual tags: %s" % "|".join(actual_tags)) # ("|".join(actual_tags))
#     print ("... Predicted tags: %s" % ("|".join(pred_tags)))    
    print ("... Predicted tags:", list(map(lambda x: (unmark_tag(
                                      x[0]), x[1]), pred_tags)))
    print ("""
===========================================================================================
    """)
# print(test_sents) # train_sents

TEXT: В Южной Корее в аэропорту Сеула задержаны граждане России. Точной информации пока нет, однако по предварительной версии они нарушили миграционное законодательство. Ситуацию прокомментировала официальный представитель российского МИДа Мария Захарова.
... Actual tags: Даниил Грибков
... Predicted tags: [('Густые и шелковистые', 0.7834113240242004), ('На экране. 10 день', 0.7168241143226624), ('На экране. 1 день', 0.7162507772445679), ('Незлоб. Лучшее', 0.7041584849357605), ('День йоги', 0.6916750073432922)]

    
TEXT: Dukascopy запустил Jforex 3
Супер ограничения FCA
Акции британцев обвалились

Watch Dukascopy TV in your language: https://www.youtube.com/user/dukascopytv
用您的语言观看杜高斯贝电视: http://www.youtube.com/user/dukascopytvchinese
Miren Dukascopy TV en su idioma: http://www.youtube.com/user/dukascopytvspanish
Schauen Sie Dukascopy TV in Ihrer Sprache: http://www.youtube.com/user/dukascopytvgerman
Regardez la Dukascopy TV dans votre langue: http://www.youtube.com/user/dukascopytvf

In [6]:
# model.save_word2vec_format('model.d2v', doctag_vec=True, word_vec=False)
model.save('model.d2v')
# tokenize_tags("йцу|dsd df аыв")
# model_d2v_TEST = Doc2Vec.load('model.d2v')